In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import ListedColormap

from tapm import miscellanea as msc
#from miscellanea import *
from tapm import utils

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.


# Parameters of the system and basic function definitions

# Final Figure

In [2]:
results = np.load("../results/min_PrEP.npy", allow_pickle=True).item()

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

# Assuming 'results' is defined
Hs = np.linspace(0, 0.2, 101)
Ps = np.linspace(0, 1, 201)

# Define the specific values for lambda_P and beta_STI
lambda_P_values = np.linspace(4/360, 0, 20001)[::100]  # Downsample for speed
beta_STI_values = np.array([7]) * 0.0016

def plot_3d(row_index=0):
    global results, Ps  

    x_values = []
    y_values = []
    z_values = []

    # Collect data for the main 3D scatter plot
    for (beta_STI, lambda_P), result in results.items():
        if beta_STI == 7 * 0.0016:
            x_values.extend(Ps)
            if row_index < result.shape[0]:
                y_values.extend(result[row_index, :])  
            else:
                y_values.extend(np.zeros_like(Ps))  
            z_values.extend([lambda_P] * len(Ps))

    x_values = np.array(x_values)
    y_values = np.array(y_values)
    z_values = np.array(z_values)
    
    # Create figure
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Main scatter plot
    sc = ax.scatter(x_values, z_values, y_values, c=y_values, cmap='viridis')
    
    # Labels and settings
    ax.set_xlabel('P')
    ax.set_ylabel('$\\lambda_P$')
    ax.set_zlabel('New cases')
    ax.set_xticks([0, 0.5, 1])
    ax.set_xticklabels(["0", "0.5", "1"])
    ax.set_yticks([0/360, 1/360, 2/360, 3/360, 4/360])
    ax.set_yticklabels(["0/yr", "1/yr", "2/yr", "3/yr", "4/yr"])
    ax.set_title('3D Plot of Results')

    # Function to calculate min PrEP uptake dynamically
    def calculate_min_PrEP_uptake_values(beta_STI_values, lambda_P_values, results, Ps, row_index):
        min_PrEP_uptake_values = {}
        max_Y = []

        for beta_STI in beta_STI_values:
            min_uptakes_for_beta = []
            for lambda_P in lambda_P_values:
                if (beta_STI, lambda_P) in results:  # Check if key exists
                    data = results[(beta_STI, lambda_P)]
                    if row_index < data.shape[0]:  # Ensure valid index
                        Y = data[row_index, :] 
                        peak_index = np.argmax(Y)
                        min_PrEP_uptake = Ps[peak_index]
                        min_uptakes_for_beta.append(min_PrEP_uptake)
                        max_Y.append(Y[peak_index])  
                    else:
                        min_uptakes_for_beta.append(0)
                        max_Y.append(0)
                else:
                    min_uptakes_for_beta.append(0)
                    max_Y.append(0)
            
            min_PrEP_uptake_values[beta_STI] = np.array(min_uptakes_for_beta)
            
        return min_PrEP_uptake_values, max_Y

    # Compute min PrEP uptake values dynamically
    min_PrEP_uptake_values, max_Y = calculate_min_PrEP_uptake_values(beta_STI_values, lambda_P_values, results, Ps, row_index)

    # Scatter plot for additional points with conditional coloring
    for beta_STI in beta_STI_values:
        points_y = min_PrEP_uptake_values[beta_STI]
        colors = ['red' if p < 1 else 'black' for p in points_y]  # Color based on condition
        ax.scatter(points_y, np.array(lambda_P_values), np.array(max_Y), c=colors, marker='o', label="Min PrEP Uptake")

    ax.legend()
    plt.show()

# Create the interactive slider
interact(plot_3d, row_index=(0, 100, 1))


interactive(children=(IntSlider(value=0, description='row_index'), Output()), _dom_classes=('widget-interact',…

<function __main__.plot_3d(row_index=0)>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

# Assuming 'results' is defined
Hs = np.linspace(0, 0.2, 101)
Ps = np.linspace(0, 1, 201)

# Define the specific values for lambda_P and beta_STI
lambda_P_values = np.linspace(4/360, 0, 20001)[::10]  # Downsample for speed
beta_STI_values = np.array([7]) * 0.0016

def plot_3d(row_index=0):
    global results, Ps  

    x_values = []
    y_values = []
    z_values = []

    # Collect data for the main 3D scatter plot
    for (beta_STI, lambda_P), result in results.items():
        if beta_STI == 7 * 0.0016:
            x_values.extend(Ps)
            if row_index < result.shape[0]:
                y_values.extend(result[row_index, :])  
            else:
                y_values.extend(np.zeros_like(Ps))  
            z_values.extend([lambda_P] * len(Ps))

    x_values = np.array(x_values)
    y_values = np.array(y_values)
    z_values = np.array(z_values)
    
    # Create figure
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Labels and settings
    ax.set_xlabel('P')
    ax.set_ylabel('$\\lambda_P$')
    ax.set_zlabel('New cases')
    ax.set_xticks([0, 0.5, 1])
    ax.set_xticklabels(["0", "0.5", "1"])
    ax.set_yticks([0/360, 1/360, 2/360, 3/360, 4/360])
    ax.set_yticklabels(["0/yr", "1/yr", "2/yr", "3/yr", "4/yr"])
    ax.set_title('3D Plot of Results')

    # Function to calculate min PrEP uptake dynamically
    def calculate_min_PrEP_uptake_values(beta_STI_values, lambda_P_values, results, Ps, row_index):
        min_PrEP_uptake_values = {}
        max_Y = []

        for beta_STI in beta_STI_values:
            min_uptakes_for_beta = []
            for lambda_P in lambda_P_values:
                if (beta_STI, lambda_P) in results:  # Check if key exists
                    data = results[(beta_STI, lambda_P)]
                    if row_index < data.shape[0]:  # Ensure valid index
                        Y = data[row_index, :] 
                        peak_index = np.argmax(Y)
                        min_PrEP_uptake = Ps[peak_index]
                        min_uptakes_for_beta.append(min_PrEP_uptake)
                        max_Y.append(Y[peak_index])  
                    else:
                        min_uptakes_for_beta.append(0)
                        max_Y.append(0)
                else:
                    min_uptakes_for_beta.append(0)
                    max_Y.append(0)
            
            min_PrEP_uptake_values[beta_STI] = np.array(min_uptakes_for_beta)
            
        return min_PrEP_uptake_values, max_Y

    # Compute min PrEP uptake values dynamically
    min_PrEP_uptake_values, max_Y = calculate_min_PrEP_uptake_values(beta_STI_values, lambda_P_values, results, Ps, row_index)

    # Scatter plot for additional points with conditional coloring
    for beta_STI in beta_STI_values:
        points_y = min_PrEP_uptake_values[beta_STI]
        colors = ['red' if p < 1 else 'black' for p in points_y]  # Color based on condition
        ax.scatter(points_y, np.array(lambda_P_values), np.array(max_Y), c=colors, marker='o', label="Min PrEP Uptake")

        # Add vertical lines parallel to X and Y axes
        for px, py, pz in zip(points_y, lambda_P_values, max_Y):
            ax.plot([px, px], [py, py], [0, pz], color='gray', linestyle='dashed', alpha=0.5)  # Z-axis line
            ax.plot([px, px], [0, py], [pz, pz], color='gray', linestyle='dashed', alpha=0.5)  # Y-axis line
    
    ax.legend()
    plt.show()

# Create the interactive slider
interact(plot_3d, row_index=(0, 100, 1))


interactive(children=(IntSlider(value=0, description='row_index'), Output()), _dom_classes=('widget-interact',…

<function __main__.plot_3d(row_index=0)>